In [19]:
#pip install -q pypdf
#pip install torch
#pip install -q transformers
#pip -q install sentence-transformers
#pip install -q llama-index

zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory
zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory
zsh:1: /Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/bin/pip: bad interpreter: /Users/hannes/Documents/Programming/GitHub/GenIpA/.conda/bin/python: no such file or directory


In [31]:

pip install  llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hannes/Documents/Programming/GitHub/GenIpA/GenIpa/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
pip install llama_index.llms

ERROR: Could not find a version that satisfies the requirement llama_index.llms (from versions: none)
ERROR: No matching distribution found for llama_index.llms
Note: you may need to restart the kernel to use updated packages.


In [40]:
import torch

import llama_index

In [45]:
from llama_index.llms import llama_cpp

In [ ]:
llama_index.llms

In [49]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

total size (MB): 7365.83


7025it [03:49, 30.60it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/hannes/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32             

ValueError: Failed to create llama_context

In [60]:
dir(llama_index.llms)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'llama_cpp']